## **Part 3: Estimation**

In [1]:
# Plotting
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio
#pio.renderers.default = "notebook+pdf"
#pio.templates.default = "plotly_dark"

# Utilities
import numpy as np
from tqdm import tqdm

### **Task 12**

In [2]:
num_women = 1000

# double check if this is the right Q matrix
Q = np.array([[0, 0.0025, 0.00125, 0, 0.001],
              [0, 0, 0, 0.002, 0.005],
              [0, 0, 0, 0.003, 0.005],
              [0, 0, 0, 0, 0.009],
              [0, 0, 0, 0, 0]])

for i in range(Q.shape[0]):
    Q[i,i] = -np.sum(Q[i,:])

print(Q)

[[-0.00475  0.0025   0.00125  0.       0.001  ]
 [ 0.      -0.007    0.       0.002    0.005  ]
 [ 0.       0.      -0.008    0.003    0.005  ]
 [ 0.       0.       0.      -0.009    0.009  ]
 [ 0.       0.       0.       0.      -0.     ]]


In [3]:
def changeState(Q, state, currentTime, endTime):
    if state == 4:
        return endTime, 4
    qii = Q[state,state]
    if qii == 0:
        return endTime, state
    time = np.random.exponential(-(1/qii))
    if currentTime + time > endTime:
        return endTime - currentTime, state
    # Determine the next state
    ps = np.copy(Q[state, :])
    ps[state] = 0
    ps /= -qii
    nextState = np.random.choice(range(5), p=ps)
    return time, nextState

def simulateWoman(Q, startingState, endTime):
    state = startingState
    time = 0
    times = [0]
    states = [state]
    while state != 4 and time < endTime:
        dt, state = changeState(Q, state, time, endTime)
        time += dt
        times.append(time)
        states.append(state)
    return np.array(times), np.array(states)

def simulateWomen(Q, n, startingStates, endTime):
    women = []
    for i in range(n):
        times, states = simulateWoman(Q, startingStates[i], endTime)
        women.append((times, states))
    return women

endTime = np.inf
startingStates = np.zeros(num_women, dtype=int)
women = simulateWomen(Q, num_women, startingStates, endTime)


In [4]:
max_time = 0
for i in range(num_women):
    temp = women[i][0][-1]

    if temp > max_time:
        max_time = temp

time_steps = np.ceil(max_time/48)

Y = np.zeros((num_women, int(time_steps+1)), dtype=int)

for i in range(num_women):
    for j in range(int(time_steps+1)):
        true_indices = np.where([women[i][0] <= 48*j])[1]
        if true_indices.size == 0:
            Y[i,j] = 0
        else:
            Y[i,j] = women[i][1][true_indices.max()]


In [11]:
vector = np.zeros((5, int(time_steps+1)))
for i in range(5):
    for j in range(int(time_steps+1)):
        vector[i,j] = np.sum(Y[:,j] == i)

stateDescs = ['No Cancer', 'Local Recurrence', 'Distant Metastasis', 'Both Local and Distant', 'Death']

fig = go.Figure()
for i in range(5):
    fig.add_trace(go.Scatter(x=48*np.arange(time_steps+1), y=vector[i,:], mode='lines', name=stateDescs[i]))
# add desription of x and y axis
fig.update_layout(
    xaxis_title="Time (months)",
    yaxis_title="Number of women in state"
)
fig.show()

### **Task 13**

In [16]:
def est_Q(Q_0, Y):
    Q = Q_0
    err = 1

    time_steps = Y.shape[1]
    num_women = Y.shape[0]

    k = 0

    while np.abs(err) > 1e-3:

        k = k + 1
        print(f"Iteration: {k}")

        N = np.zeros((5,5))
        S = np.zeros((5))

        Q_old = np.copy(Q)

        for i in tqdm(range(num_women)):
            for j in range(1, int(time_steps)):
                while True:
                    women = simulateWomen(Q_old, 1, [Y[i,j-1]], 48)

                    Y_new_state = women[0][1]
                    Y_new_time = women[0][0]

                    if Y_new_state[-1] == Y[i,j]:
                        for k in range(1,len(Y_new_state)):
                            N[Y_new_state[k-1],Y_new_state[k]] += 1
                            S[Y_new_state[k]] += Y_new_time[k]
                        break

        Q = np.zeros((5,5))

        for i in range(5):
            for j in range(5):
                if i != j:
                    Q[i,j] = N[i,j]/S[i]

            Q[i,i] = -np.sum(Q[i,:])

        err = np.linalg.norm(Q_old - Q)

    return Q

def generate_matrix():
    # Generate a 5x5 matrix with random positive elements
    matrix = np.random.rand(5, 5)
    matrix /= 100

    # Make the diagonal elements the negative sum of the remaining row elements
    np.fill_diagonal(matrix, -matrix.sum(axis=1) + np.diag(matrix))

    return matrix

Q_01 = generate_matrix()
Q_02 = np.copy(Q)

Q2 = est_Q(Q_02, Y)
Q1 = est_Q(Q_01, Y)


Iteration: 1


100%|██████████| 1000/1000 [00:01<00:00, 571.80it/s]


Iteration: 2


100%|██████████| 1000/1000 [00:01<00:00, 787.40it/s]


Iteration: 1


...

100%|██████████| 1000/1000 [00:01<00:00, 914.12it/s]


Iteration: 2


100%|██████████| 1000/1000 [00:00<00:00, 1116.58it/s]


Iteration: 2


100%|██████████| 1000/1000 [00:00<00:00, 1242.52it/s]


In [17]:
print("True matrix")
print(Q)
print("\n")

print("Random matrix")
print(Q_01)
print("\n")

print("From random matrix")
with np.printoptions(suppress=True):
    print(np.round(Q1, 5))
print("\n")

print("From true matrix")
print(np.round(Q2, 5))
print("\n")

print(np.linalg.norm(Q1 - Q2))

True matrix
[[-0.00475  0.0025   0.00125  0.       0.001  ]
 [ 0.      -0.007    0.       0.002    0.005  ]
 [ 0.       0.      -0.008    0.003    0.005  ]
 [ 0.       0.       0.      -0.009    0.009  ]
 [ 0.       0.       0.       0.      -0.     ]]


Random matrix
[[-0.01910434  0.00414031  0.00967065  0.00054224  0.00475114]
 [ 0.00606369 -0.02589853  0.00991725  0.00569917  0.00421841]
 [ 0.00600835  0.0098022  -0.02914006  0.00482201  0.0085075 ]
 [ 0.00077268  0.00660384  0.00678534 -0.01984402  0.00568216]
 [ 0.00343321  0.00278938  0.0071282   0.00459005 -0.01794084]]


From random matrix
[[-0.00576  0.00298  0.00141  0.00007  0.0013 ]
 [ 0.      -0.00606  0.00004  0.00179  0.00423]
 [ 0.00003  0.      -0.00681  0.00231  0.00448]
 [ 0.       0.       0.      -0.00735  0.00735]
 [ 0.       0.       0.       0.      -0.     ]]


From true matrix
[[-0.00576  0.00306  0.0015   0.       0.0012 ]
 [ 0.      -0.0062   0.       0.00179  0.00441]
 [ 0.       0.      -0.00708  0.00253 